<a href="https://colab.research.google.com/github/pallottom/planets/blob/master/ploting_on_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLOTTING ON MAPS

#### QUESTION

How could we have changed/improved the animated plot from week 1?

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # little extra step to allow colab to access my google drive

In [ ]:
!pip install geopandas==0.8.0
!pip install folium

In [ ]:
# move to colb directory
cd drive/MyDrive/Colab\ Notebooks

In [ ]:
# check that I am actually in the right directory
ls

#### READ DATA

We need to import two types of data:
1. the values we want to plot (numerical data, ex. .csv );
2. the shape of the country/lands/veedel/area we want to visualize (shape file).

In [ ]:
# read historical temperature data
data = 'data/all_country_temp_data_CLEAN.csv'
import pandas as pd
ddf = pd.read_csv(data)

In [ ]:
ddf

In [ ]:
# read geographical data (what's a shape file?)
shapefile= 'data/ne_110m_admin_0_countries.shp'
import geopandas as gpd
gdf = gpd.read_file(shapefile)

In [ ]:
gdf

#### DATASET PREPARATION

In [ ]:
# gdf is a bit too big and chaotic, let's clean it up!
gdf = gpd.read_file(shapefile)[['ADMIN', 'geometry']] # import only ADMIN and geometry columns
gdf.columns = ['country', 'geometry'] # rename the columns
# ddf we want to group ausung the mean of monthy_anomaly (one value for year)
ddf = ddf.groupby(['country', 'year'])[['monthly_anomaly']].mean().reset_index() #get the mean of the year

We need ONE only dataset with: country name, geometry of the contry and 'monthly anomaly'

In [ ]:
# merge gdf and ddf
df = pd.merge(left=gdf, right=ddf, how='left', on='country')

In [ ]:
# convert the data relative to year 2000 to JSON data
df_2000 = df[df['year'] == 2000] 
json_2000 = df_2000.to_json() 

#### GENERATE THE CHOROPLETH MAP

In [ ]:
import folium
tiles = folium.Choropleth(
            geo_data=json_2000,                 # geojson string that includes the geo data (for the year 2000)
            name="2000 Data",                   # just a title
            data=df_2000,                      # dataframe that includes the data (for the year 2000)
            columns=['country','monthly_anomaly'], # names of the columns to include from the dataframe
            key_on='properties.country',        # name of JSON key within the "properties" value that contains country names
            fill_color='YlGnBu',                # play around with the rest of the aesthetic options
            nan_fill_color='black',         
            fill_opacity=0.8,
            line_opacity=0.1,
            legend_name='Monthly Temperature Anomaly', # Name of variable for the colour bar.
            highlight=True,
        )

#### CREATE A WHITE BASELINE MAP

In [ ]:
my_map = folium.Map(location=[52.54, 13.36], #coordinates of the center of the map
                zoom_start=2,
                tiles='CartoDB positron') #try out the other options!

In [ ]:
my_map

#### MERGE AND VISUALIZE THE TWO MAPS

In [ ]:
tiles.add_to(my_map) #add the layer to the blank canvas / figure
my_map

In [ ]:
basemap.save('mymap.html')

#### MORE INFO

- [folium](https://python-visualization.github.io/folium/)
- [geopandas and choropleph](https://geopandas.org/en/stable/gallery/choro_legends.html)
- [JSON](https://en.wikipedia.org/wiki/JSON)
- [Plotly: another graphing tool](https://plotly.com/python/)
- [comparison folium Plotly](https://mari-galdina.medium.com/mapping-data-plotly-vs-folium-1fa2747c9d35)